##  Huggett(1996) model
#### This version has certain lifetimes.

In [1]:
using Optim
using Interpolations
using QuantEcon
using Roots

In [2]:
# Discretization of the labour productivity shock with Tauchen. 
function tauchen1(N::Integer, ρ::Real, σ::Real, μ::Real=0.0, n_std::Integer=3)

  std_norm_cdf{T <: Real}(x::T) = 0.5 * erfc(-x/sqrt(2))
  std_norm_cdf{T <: Real}(x::Array{T}) = 0.5 .* erfc(-x./sqrt(2))

    # Get discretized space
    a_bar = n_std * sqrt(σ^2 / (1 - ρ^2))
    y = linspace(-a_bar, a_bar, N)
    d = y[2] - y[1]

    # Get transition probabilities
    Π = zeros(N, N)
    for row = 1:N
        # Do end points first
        Π[row, 1] = std_norm_cdf((y[1] - ρ*y[row] + d/2) / σ)
        Π[row, N] = 1 - std_norm_cdf((y[N] - ρ*y[row] - d/2) / σ)

        # fill in the middle columns
        for col = 2:N-1
            Π[row, col] = (std_norm_cdf((y[col] - ρ*y[row] + d/2) / σ) -
                           std_norm_cdf((y[col] - ρ*y[row] - d/2) / σ))
        end
    end

    # NOTE: I need to shift this vector after finding probabilities
    #       because when finding the probabilities I use a function
    #       std_norm_cdf that assumes its input argument is distributed
    #       N(0, 1). After adding the mean E[y] is no longer 0, so
    #       I would be passing elements with the wrong distribution.
    #
    #       It is ok to do after the fact because adding this constant to each
    #       term effectively shifts the entire distribution. Because the
    #       normal distribution is symmetric and we just care about relative
    #       distances between points, the probabilities will be the same.
    #
    #       I could have shifted it before, but then I would need to evaluate
    #       the cdf with a function that allows the distribution of input
    #       arguments to be [μ/(1 - ρ), 1] instead of [0, 1]

    yy = y .+ μ / (1 - ρ) # center process around its mean (wbar / (1 - rho)) in new variable

    # renormalize. In some test cases the rows sum to something that is 2e-15
    # away from 1.0, which caused problems in the MarkovChain constructor
    Π = Π./sum(Π, 2)

    return Π, yy
end

tauchen1 (generic function with 3 methods)

Define the Model type and initialize parameter values

In [3]:
type Huggett
    beta::Float64 # discount factor
    sigma::Float64 # CRRA coefficient
    A::Float64 # aggregate productivity
    alpha::Float64 # capital share of output
    delta::Float64 # depreciation rate
    R::Int64 # Retirement age
    N::Int64 # Death age
    n::Float64 # growth rate
    gamma::Float64 # persistence of earnings process
    sig_e::Float64 # std dev of earnings process
    theta::Float64 # social security tax on labour earnings
    agrid::Vector{Float64} # asset grid
    Na::Int64
    zgrid::Vector{Float64} # earnings shock grid
    Pi::Matrix{Float64}
    Nz::Int64
    KY::Float64 # capital output ratio
    tau::Float64 # income tax rate
    mu::Vector{Float64}
    earnings::Vector{Float64} # mean earnings by age 
    e_mat::Matrix{Float64} # e(z,t) matrix (product of earnings and zgrid)
end

function Huggett(;beta=0.99, sigma=1.5, A=0.895944,alpha=0.36,delta=0.06,R=46,N=79,n=0.012,gamma=0.96,sig_e=0.212,theta=0.10,KY=3.0)
    
    Pi,lnzgrid=tauchen1(18,gamma,sig_e,0.0,4) # 18 is the grid size and 4 is the # of std dev out
    zgrid=exp(lnzgrid)
    agrid=linspace(0.0,20,40)
    Nz=length(zgrid)
    Na=length(agrid)
    tau=0.195/(1-delta*(KY))
   
    # age distribution
    x=(1/(1+n))*ones(N)
    y=(-1)*linspace(1,79,79)
    mu=x.^-y 
    
    #earnings
    earnings = [0.0911, 0.1573, 0.2268, 0.2752, 0.3218, 0.3669, 0.4114, 0.4559, 0.4859, 0.5164, 0.5474, 0.5786, 0.6097, 0.6311, 0.6517, 0.6711, 0.6893, 0.7060, 0.7213, 0.7355, 0.7489, 0.7619, 0.7747, 0.7783, 0.7825, 0.7874, 0.7931, 0.7994, 0.7923, 0.7850, 0.7771, 0.7679, 0.7567, 0.7351, 0.7105, 0.6822, 0.6500,
    0.6138, 0.5675, 0.5183, 0.4672, 0.3935, 0.3239, 0.2596, 0.1955, 0.1408, 0.0959, 0.0604, 0.0459, 0.0342, 0.0246, 0.0165, 0.0091, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
    
    e_mat=kron(transpose(earnings),zgrid)
    
    Huggett(beta,sigma,A,alpha,delta,R,N,n,gamma,sig_e,theta,agrid,Na,zgrid,Pi,Nz,KY,tau,mu,earnings,e_mat)
    
    
end

Huggett

Write functions for calculating utility, wage, and interest rate.

In [4]:
function w_r(alpha::Float64,A::Float64,K::Float64,L::Float64,delta::Float64)
    w=(1-alpha)*A*(K^alpha)*(L^(-alpha))
    r=alpha*A*(K^(alpha-1))*(L^(1-alpha))-delta
    
    return w,r
end

function util(c::Float64,sigma::Float64)
    u=(c^(1-sigma))/(1-sigma)
    if c<=0.0 #inada
        u=-1e8
    end
    return u
end


util (generic function with 1 method)

Function for solving for the optimal asset holding.

Solve for the policy function by backward induction.

In [5]:
function find_a(bgrid,V::Array{Float64},i_t::Int64,i_z::Int64,hm::Huggett,a::Float64,e::Float64,r::Float64,w::Float64,b::Float64)
    function opt_a(aprime)
        c=a*(1+r*(1-hm.tau))+(1-hm.theta-hm.tau)*e*w+bgrid[i_t]-aprime
        interpV=zeros(hm.Nz)
        
        for zp=1:hm.Nz
            itp=interpolate((hm.agrid,),vec(V[:,zp,i_t+1]),Gridded(Linear()))
            extp=extrapolate(itp,Linear())
            interpV[zp]=extp[aprime]
        end
        
        exp_valre=transpose(hm.Pi[i_z,:])*interpV
        exp_val=exp_valre[1]
        
        utility_flow=util(c,hm.sigma)
        Vnew=(utility_flow+hm.beta*exp_val)*(-1.0)

        return Vnrw
    end

    results=optimize(opt_a,0.0,100.0)
    aprime=Optim.minimizer(results)
    V=(-1.0)*Optim.minimum(results)
    c=a*(1+r*(1-hm.tau))+(1-hm.theta-hm.tau)*e*w+b-aprime
    return aprime,V
end

find_a (generic function with 1 method)

The labour endowment is $e(z,t)=exp(z_t+\bar{y_t})$ where $\bar{y_t}$ is the mean log endowment of agents of age $t$

In [8]:
function hhdecision(hm::Huggett,K::Float64)
    beta,sigma,A,alpha,delta,R,N,n,gamma,sig_e,theta,agrid,Na,zgrid,Pi,Nz,KY,tau,mu,earnings,e_mat=hm.beta,hm.sigma,hm.A,hm.alpha,hm.delta,hm.R,hm.N,hm.n,hm.gamma,hm.sig_e,hm.theta,hm.agrid,hm.Na,hm.zgrid,hm.Pi,hm.Nz,hm.KY,hm.tau,hm.mu,hm.earnings,hm.e_mat
    
    # calculate L
    Y=(1/3)*K  
    L=(Y/(A*(K^alpha)))^(1/(1-alpha))
    # measure old
    w,r=w_r(alpha,A,K,L,delta)
    measure_old=sum(mu[R:end])
    V=zeros(Na,Nz,N+1) # value for the agents by assets, earnings shock, and age
    aprime=similar(V) # asset choice
    cons=similar(V)
    bgrid=zeros(N+1)
    bgrid[R:N]=theta*w*L/measure_old*ones(N-R+1)
    
    for i_t=1:N
        for (i_z,z) in enumerate(zgrid)
            for (i_a,a) in enumerate(agrid)
                aprime[i_a,i_z,N+1-i_t],V[i_a,i_z,N+1-i_t],cons[i_a,i_z,N+1-i_t]=find_a(bgrid,V,i_t,i_z,hm,a,e_mat[i_z,i_t],r,w,bgrid[N+1-i_t])
            end 
        end
    end
    
    return V,aprime,cons
end


hhdecision (generic function with 1 method)

In [10]:
K=1.7
hm=Huggett()
#V,aprime,cons=hhdecision(hm,K)

Huggett(0.99,1.5,0.895944,0.36,0.06,46,79,0.012,0.96,0.212,0.1,[0.0,0.512821,1.02564,1.53846,2.05128,2.5641,3.07692,3.58974,4.10256,4.61538  …  15.3846,15.8974,16.4103,16.9231,17.4359,17.9487,18.4615,18.9744,19.4872,20.0],40,[0.0483847,0.0690953,0.0986708,0.140906,0.201219,0.287349,0.410345,0.585989,0.836816,1.19501,1.70652,2.43697,3.48009,4.96971,7.09694,10.1347,14.4728,20.6677],[0.606 0.368387 … 0.0 0.0; 0.0893873 0.542228 … 0.0 0.0; … ; 3.02424e-144 3.49192e-126 … 0.542228 0.0893873; 9.73451e-163 1.68512e-143 … 0.368387 0.606],18,3.0,0.23780487804878048,[0.988142,0.976425,0.964847,0.953406,0.942101,0.93093,0.919891,0.908983,0.898205,0.887554  …  0.433874,0.42873,0.423646,0.418623,0.413659,0.408754,0.403907,0.399117,0.394385,0.389708],[0.0911,0.1573,0.2268,0.2752,0.3218,0.3669,0.4114,0.4559,0.4859,0.5164  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.00440785 0.00761091 … 0.0 0.0; 0.00629458 0.0108687 … 0.0 0.0; … ; 1.31847 2.27657 … 0.0 0.0; 1.88283 3.25103 … 0.0 0.0])